In [1]:
from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from numpy.linalg import lstsq
from ngsolve import * 
from ngsolve.webgui import Draw

dg = DataGenerator(maxh = (0.1, 0.3))

medium = MediumGenerator(cosine_func) # MediumGenerator(disk_func)

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)



params  = [{"type": 0, "x": 0.6 ,"y": 0.4, "r": 0.3, "v": 1}, 
           {"type": 0, "x": 0.7, "y": -0.6, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.8, "y": -0.3, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.5, "y": 0.5, "r": 0.3, "v": 1}]

permittivity  = medium.generate(params)


Mesh generation took 0.12120610900456086 seconds


In [2]:
# direction_angles = np.linspace(0, 2 * np.pi, 32, endpoint=False)

# cauchy_data = dg.generate_cauchy_data(2 * pi, direction_angles, permittivity)

In [3]:
n_dir = 8

direction_angles = np.linspace(0, 2 * np.pi, n_dir, endpoint=False)

a = BilinearForm(dg.fes)
a += (0.025 * grad(dg.u) * grad(dg.v)) * (IfPos((x)**2 + (y)**2 - (1.5) **2,
                    0,
                    1)) * dx + dg.u * dg.v * dx
a.Assemble()

A = Matrix(len(direction_angles) ** 2, dg.fes.ndof, complex=True)
b = Vector(len(direction_angles) ** 2, complex=True)

freq = 0.0

for iter in range(210):
    print(iter)

    l = 0
    freq += 0.1 *  (1 / int(freq + 1))

    print(freq)

    direction_angles = np.linspace(0, 2 * np.pi, n_dir, endpoint=False)

    direction_angles += np.random.random()*2*np.pi

    for i in range(n_dir):

        kx = 2 * pi * freq * cos(direction_angles[i])
        ky = 2 * pi * freq * sin(direction_angles[i])

        psi1 = CF((exp(1j * kx * x) * exp(1j * ky * y)))

        u_scat = dg.solve(kx, ky, permittivity)
        uB_scat0 = dg.solve(kx, ky, background_permittivity)

        r_B = 1.5
        theta_B = np.linspace(0, 2* pi, 361)[0:-1]

        pts = [(r_B * cos(theta_B[i]), r_B * sin(theta_B[i])) for i in range(len(theta_B))]


        u_scat_vals = [u_scat(dg.mesh(*pt)) for pt in pts]
        grad_u_scat_vals = [ (grad(u_scat)(dg.mesh(*pt))[0] * pt[0] + grad(u_scat)(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


        for j in range(n_dir):
            p_kx = 2 * pi * freq *  cos(direction_angles[j])
            p_ky = 2 * pi * freq * sin(direction_angles[j])

            psi2 = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))

            uB_scat = dg.solve(p_kx, p_ky, background_permittivity)


            # dg.save(u_scat, "scatter", "data/scatter.mat")
            grad_uB_scat = grad(uB_scat)


            uB_scat_vals = [uB_scat(dg.mesh(*pt)) for pt in pts]
            grad_uB_scat_vals = [ (grad_uB_scat(dg.mesh(*pt))[0] * pt[0] + grad_uB_scat(dg.mesh(*pt))[1] * pt[1]) / r_B for pt in pts ]


            # define domain B as a disk of radius 1.5
            # generate Cauchy data

            psi_test1 = CF((exp(1j * kx * x) * exp(1j * ky * y)))
            grad_psi_test_X1 = psi_test1.Diff(x)
            grad_psi_test_Y1 = psi_test1.Diff(y)

            psi_vals1 = [psi_test1(dg.mesh(*pt)) for pt in pts]
            grad_psi_vals1 = [ (grad_psi_test_X1(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y1(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]



            psi_test2 = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))
            grad_psi_test_X2 = psi_test2.Diff(x)
            grad_psi_test_Y2 = psi_test2.Diff(y)

            psi_vals2 = [psi_test2(dg.mesh(*pt)) for pt in pts]
            grad_psi_vals2 = [ (grad_psi_test_X2(dg.mesh(*pt)) * pt[0] + grad_psi_test_Y2(dg.mesh(*pt)) * pt[1]) / r_B  for pt in pts ]


            # s = sum( [grad_psi_vals[i] * uB_scat_vals[i] - grad_uB_scat_vals[i] * psi_vals[i] for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)

            # print(s)

            # s = sum( [(grad_uB_scat_vals[i] + grad_psi_vals2[i]) * (u_scat_vals[i]+ psi_vals1[i]) - (grad_u_scat_vals[i]+ grad_psi_vals1[i]) * (uB_scat_vals[i] + psi_vals2[i]) for i in range(len(pts))] ) * (2 * pi * r_B / len(pts) ) / (kx**2 + ky**2)


            true_val = Integrate( (permittivity - background_permittivity) * (uB_scat + psi2) *  (u_scat + psi1), dg.mesh)
            cur_val =  Integrate( (permittivity - background_permittivity) * (uB_scat + psi2) *  (uB_scat0 + psi1), dg.mesh)

            # print(i,j, true_val, cur_val, true_val - cur_val)

            tmp = dg.fes.TrialFunction()
            tmp2 = dg.fes.TestFunction()

            linear_form = LinearForm(dg.fes)

            linear_form += tmp2 * (uB_scat + psi2) *  (u_scat + psi1) * (IfPos((x)**2 + (y)**2 - (1.5) **2,
                    0,
                    1))  * dx

            linear_form.Assemble()

            A.NumPy()[l, :] = linear_form.vec.FV().NumPy()
            b[l] = true_val
            l += 1
    
    v = lstsq(A.NumPy(),  b.NumPy(), rcond=None)[0]
    
    permittivity_update =  GridFunction(dg.fes)
    permittivity_update.vec.data = v
    linear_form = LinearForm(dg.fes)
    linear_form += permittivity_update * dg.v *  (IfPos((x)**2 + (y)**2 - (1.5) **2,
                    0,
                    1))  * dx
    linear_form.Assemble()
    # permittivity_update.vec.data = v
    permittivity_update.vec.data = a.mat.Inverse(freedofs=dg.fes.FreeDofs(),
                                        inverse="sparsecholesky") * linear_form.vec



    background_permittivity = background_permittivity + permittivity_update

    scene = Draw(background_permittivity, dg.mesh)
    clear_output() # redraw the scene with a new height
    scene.Draw(height="3vh")

WebGuiWidget(layout=Layout(height='3vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'spe…

6
0.7
Solving took 0.15019181196112186 seconds
Solving took 0.14054998802021146 seconds
Solving took 0.14090515405405313 seconds
Solving took 0.13095791899831966 seconds
Solving took 0.13442173798102885 seconds
Solving took 0.14085872803116217 seconds
Solving took 0.13489157397998497 seconds
Solving took 0.13719652697909623 seconds
Solving took 0.14448049699421972 seconds
Solving took 0.1365296210278757 seconds
Solving took 0.12512631801655516 seconds
Solving took 0.1341938230325468 seconds
Solving took 0.13425418396946043 seconds
Solving took 0.13921345199923962 seconds
Solving took 0.13661125401267782 seconds
Solving took 0.14550416101701558 seconds
Solving took 0.1768629919970408 seconds
Solving took 0.13899562298320234 seconds
Solving took 0.1322648079949431 seconds
Solving took 0.13364715495845303 seconds
Solving took 0.12647091003600508 seconds
Solving took 0.13590681698406115 seconds
Solving took 0.13700139796128497 seconds
Solving took 0.13486344798002392 seconds
Solving took 0

KeyboardInterrupt: 

In [12]:
scene = Draw(background_permittivity, dg.mesh)
clear_output() # redraw the scene with a new height
scene.Draw(height="3vh")

2.0793851447914387